In [2]:
from sklearn.decomposition import PCA
import data_prep as dp
import matplotlib.pyplot as plt
import statistics
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import statsmodels.api as sm
import numpy as np
import pandas as pd
import scipy
import pickle

def itr(ac):
    num_tac= 6 #possible selections
    spm = 60000/620/8/num_tac #selections per minute
    itr = (np.log2(num_tac)+ ac*np.log2(ac)+ (1-ac)* np.log2((1-ac)/(num_tac-1)))*spm 
    return itr



pd.set_option('display.max_columns', 1500) 
pd.set_option('display.max_rows', 1500) 

# Amplitudes

In [98]:
def load_session_dates():
    with open(r"D:\Master\Masterarbeit\Data\date_dict.pickle", "rb") as file:
        SESS_DATES = pickle.load(file)
    return SESS_DATES

SESS_DATES = load_session_dates()
amp = pd.read_csv(r"D:\Master\Masterarbeit\Data\Amplitudes\average_amplitudes_350_600.csv")
amp["Dates"] = amp["Session"].replace(SESS_DATES)

amp.to_csv(r"D:\Master\Masterarbeit\Data\Classifier_Results\amp.csv",index=False)


# SESS3 Load Data for the sesionwise, 3 calibration runs

In [48]:
### Gathering Data of the Session wise calibration

#loading the data of the python classifiers

sess3 = pd.read_csv(r"D:\Master\Masterarbeit\Data\Classifier_Results\accuracies_sess3.csv")

# renaming sess3ions so that the Session number corresponds to the measuring day

dic = dict(zip(sess3["Session"].unique(),list(range(1,40))))
sess3["Session"] = sess3["Session"].replace(dic)

#loading the BCI2000 data from a text file

bci2k = pd.read_fwf(r"D:\Master\Masterarbeit\Data\Classifier_Results\bci2000_Sessionwise.txt",header=None)[[0,1]].to_numpy()
temp_df = pd.DataFrame(columns=["Accuracy", "Classifier", "Session", "Ep2Avg"])
temp_df["Accuracy"] = bci2k[:,1]*0.01
temp_df["Classifier"] = "BCI2000"
temp_df["Session"] =  [i for i in range(1,int(len(bci2k)/8)+1) for _ in range(8)]
temp_df["Ep2Avg"] = bci2k[:,0]
temp_df

# concatation of the dataframes and added the attribute calibration mode and ITR

sess3 = sess3.append(temp_df)
sess3 = sess3.drop(columns=["Unnamed: 0"])
sess3["Calibration"] = "sess3"
sess3["ITR"] = itr(sess3.Accuracy)
sess3



,Accuracy,Classifier,Session,Ep2Avg,Calibration,ITR
0,0.444444,MDM_res_xDawn,1,1,sess3,0.388274
1,0.222222,MDM_res_xDawn,1,2,sess3,0.009214
2,0.277778,MDM_res_xDawn,1,3,sess3,0.008769
3,0.611111,MDM_res_xDawn,1,4,sess3,1.268791
4,0.611111,MDM_res_xDawn,1,5,sess3,1.268791
...,...,...,...,...,...,...
307,0.610000,BCI2000,39,4,sess3,1.261286
308,0.610000,BCI2000,39,5,sess3,1.261286
309,0.610000,BCI2000,39,6,sess3,1.261286
310,0.610000,BCI2000,39,7,sess3,1.261286


In [50]:
sess3.to_csv(r"D:\Master\Masterarbeit\Data\Classifier_Results\sess3.csv",index=False)


# Sess1

In [51]:
### Gathering Data of the Session wise calibration

#loading the data of the python classifiers

sess1 = pd.read_csv(r"D:\Master\Masterarbeit\Data\Classifier_Results\accuracies_sess1.csv")

# renaming sess1ions so that the Session number corresponds to the measuring day

dic = dict(zip(sess1["Session"].unique(),list(range(1,40))))
sess1["Session"] = sess1["Session"].replace(dic)

#loading the BCI2000 data from a text file

bci2k = pd.read_fwf(r"D:\Master\Masterarbeit\Data\Classifier_Results\bci2000_sessionwise_singleRunCalibration.txt",header=None)[[0,1]].to_numpy()
temp_df = pd.DataFrame(columns=["Accuracy", "Classifier", "Session", "Ep2Avg"])
temp_df["Accuracy"] = bci2k[:,1]*0.01
temp_df["Classifier"] = "BCI2000"
temp_df["Session"] =  [i for i in range(1,int(len(bci2k)/8)+1) for _ in range(8)]
temp_df["Ep2Avg"] = bci2k[:,0]
temp_df

# concatation of the dataframes and added the attribute calibration mode and ITR

sess1 = sess1.append(temp_df)
sess1 = sess1.drop(columns=["Unnamed: 0"])
sess1["Calibration"] = "sess1"
sess1["ITR"] = itr(sess1.Accuracy)
sess1



,Accuracy,Classifier,Session,Ep2Avg,Calibration,ITR
0,0.222222,MDM_res_xDawn,1,1,sess1,0.009214
1,0.222222,MDM_res_xDawn,1,2,sess1,0.009214
2,0.277778,MDM_res_xDawn,1,3,sess1,0.008769
3,0.277778,MDM_res_xDawn,1,4,sess1,0.008769
4,0.444444,MDM_res_xDawn,1,5,sess1,0.388274
...,...,...,...,...,...,...
307,0.440000,BCI2000,39,4,sess1,0.371472
308,0.500000,BCI2000,39,5,sess1,0.627577
309,0.670000,BCI2000,39,6,sess1,1.699706
310,0.720000,BCI2000,39,7,sess1,2.119233


In [52]:
sess1.loc[(sess1.Ep2Avg==8)].groupby("Classifier").mean()

,Accuracy,Session,Ep2Avg,ITR
Classifier,,,,
BCI2000,0.413590,20.0,8.0,0.546379
FGMDM,0.381766,20.0,8.0,0.381634
FGMDM_res,0.443020,20.0,8.0,0.577365
FGMDM_res_xDawn,0.594017,20.0,8.0,1.575718
FGMDM_xDawn,0.618234,20.0,8.0,1.680151
LDA,0.334758,20.0,8.0,0.272277
LDA_xDawn,0.507123,20.0,8.0,0.949808
MDM,0.441595,20.0,8.0,0.812984
MDM_res,0.329060,20.0,8.0,0.351418


In [53]:
sess1.to_csv(r"D:\Master\Masterarbeit\Data\Classifier_Results\sess1.csv",index=False)


# Trans3

In [54]:
### Gathering Data of the Session wise calibration

#loading the data of the python classifiers

trans3 = pd.read_csv(r"D:\Master\Masterarbeit\Data\Classifier_Results\accuracies_trans3.csv")

# renaming trans3ions so that the Session number corresponds to the measuring day

dic = dict(zip(trans3["Session"].unique(),list(range(1,40))))
trans3["Session"] = trans3["Session"].replace(dic)

#loading the BCI2000 data from a text file

bci2k = pd.read_fwf(r"D:\Master\Masterarbeit\Data\Classifier_Results\bci2000_3session_transfer.txt",header=None)[[0,1]].to_numpy()
temp_df = pd.DataFrame(columns=["Accuracy", "Classifier", "Session", "Ep2Avg"])
temp_df["Accuracy"] = bci2k[:,1]*0.01
temp_df["Classifier"] = "BCI2000"
temp_df["Session"] =  [i for i in range(1,int(len(bci2k)/8)+1) for _ in range(8)]
temp_df["Ep2Avg"] = bci2k[:,0]
temp_df

# concatation of the dataframes and added the attribute calibration mode and ITR

trans3 = trans3.append(temp_df)
trans3 = trans3.drop(columns=["Unnamed: 0"])
trans3["Calibration"] = "trans3"
trans3["ITR"] = itr(trans3.Accuracy)
trans3



,Accuracy,Classifier,Session,Ep2Avg,Calibration,ITR
0,0.166667,MDM_res_xDawn,1,1,trans3,0.088232
1,0.222222,MDM_res_xDawn,1,2,trans3,0.009214
2,0.166667,MDM_res_xDawn,1,3,trans3,0.088232
3,0.111111,MDM_res_xDawn,1,4,trans3,0.265785
4,0.444444,MDM_res_xDawn,1,5,trans3,0.388274
...,...,...,...,...,...,...
307,0.500000,BCI2000,39,4,trans3,0.627577
308,0.560000,BCI2000,39,5,trans3,0.946660
309,0.560000,BCI2000,39,6,trans3,0.946660
310,0.610000,BCI2000,39,7,trans3,1.261286


In [55]:
trans3.to_csv(r"D:\Master\Masterarbeit\Data\Classifier_Results\trans3.csv",index=False)


# Trans1

In [56]:
### Gathering Data of the Session wise calibration

#loading the data of the python classifiers

trans1 = pd.read_csv(r"D:\Master\Masterarbeit\Data\Classifier_Results\accuracies_trans1.csv")

# renaming trans1ions so that the Session number corresponds to the measuring day

dic = dict(zip(trans1["Session"].unique(),list(range(1,40))))
trans1["Session"] = trans1["Session"].replace(dic)

#loading the BCI2000 data from a text file

bci2k = pd.read_fwf(r"D:\Master\Masterarbeit\Data\Classifier_Results\bci2000_singleSession_transfer.txt",header=None)[[0,1]].to_numpy()
temp_df = pd.DataFrame(columns=["Accuracy", "Classifier", "Session", "Ep2Avg"])
temp_df["Accuracy"] = bci2k[:,1]*0.01
temp_df["Classifier"] = "BCI2000"
temp_df["Session"] =  [i for i in range(1,int(len(bci2k)/8)+1) for _ in range(8)]
temp_df["Ep2Avg"] = bci2k[:,0]
temp_df

# concatation of the dataframes and added the attribute calibration mode and ITR

trans1 = trans1.append(temp_df)
trans1 = trans1.drop(columns=["Unnamed: 0"])
trans1["Calibration"] = "trans1"
trans1["ITR"] = itr(trans1.Accuracy)
trans1



,Accuracy,Classifier,Session,Ep2Avg,Calibration,ITR
0,0.444444,MDM_res_xDawn,1,1,trans1,0.388274
1,0.222222,MDM_res_xDawn,1,2,trans1,0.009214
2,0.277778,MDM_res_xDawn,1,3,trans1,0.008769
3,0.611111,MDM_res_xDawn,1,4,trans1,1.268791
4,0.611111,MDM_res_xDawn,1,5,trans1,1.268791
...,...,...,...,...,...,...
307,0.330000,BCI2000,39,4,trans1,0.070006
308,0.280000,BCI2000,39,5,trans1,0.010210
309,0.330000,BCI2000,39,6,trans1,0.070006
310,0.500000,BCI2000,39,7,trans1,0.627577


In [57]:
trans1.to_csv(r"D:\Master\Masterarbeit\Data\Classifier_Results\trans1.csv",index=False)


# Concatenate all results

In [58]:
all_classifier_results = sess1.append(trans1).append(trans3).append(sess3)
all_classifier_results

,Accuracy,Classifier,Session,Ep2Avg,Calibration,ITR
0,0.222222,MDM_res_xDawn,1,1,sess1,0.009214
1,0.222222,MDM_res_xDawn,1,2,sess1,0.009214
2,0.277778,MDM_res_xDawn,1,3,sess1,0.008769
3,0.277778,MDM_res_xDawn,1,4,sess1,0.008769
4,0.444444,MDM_res_xDawn,1,5,sess1,0.388274
...,...,...,...,...,...,...
307,0.610000,BCI2000,39,4,sess3,1.261286
308,0.610000,BCI2000,39,5,sess3,1.261286
309,0.610000,BCI2000,39,6,sess3,1.261286
310,0.610000,BCI2000,39,7,sess3,1.261286


In [59]:
all_classifier_results.to_csv(r"D:\Master\Masterarbeit\Data\Classifier_Results\all_classifier_results.csv")
